# Assignment 2 - Michela Sessi 777760

## Bag of Words con LBP (Local Binary Pattern)

In [2]:
import os
import cv2 as cv
from matplotlib import pyplot as plt
from skimage import feature
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import accuracy_score
import numpy as np
from time import time

Creiamo la funzione load_lbp_features che consente di caricare le immagini a partire da diverse cartelle relative alle diverse classi. La funzione permette di ottenere delle features basate su LBP (Local Binary Pattern). Una volta calcolato LBP si scompone la rappresentazione in blocchi sovrapposti e ad ogni scorrimento si calcola l'istogramma per ciascun blocco.

La funzione ci restituisce quattro oggetti X_train, X_test, y_train, y_test 

In [5]:
def load_lbp_features(base_path, 
                  maximages=500, 
                  extension='.jpg', 
                  Points=24, Radius=8, shift = 8, dims = 16):
    
    labels = []
    features = []
    for di,d in enumerate(sorted(os.listdir(base_path))):
        for fi,f in enumerate(sorted(os.listdir(base_path + d + '/'))):
            if f.endswith(extension) and fi<maximages:
                image = cv.imread(base_path + d + '/' + f, 0)            
                #lbp_features
                image1 = feature.local_binary_pattern(image, P=Points, R=Radius, method='uniform')
                feats = []
                for i in range(0,image1.shape[0],shift):
                    for j in range(0,image1.shape[1],shift):
                        hist = np.bincount(image1[i:i+dims,j:j+dims].flatten().astype(int), minlength=Points+2)
                        feats.append(hist)
                cur_features = np.array(feats)
            
                features.append(cur_features)
                labels.append(di)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, shuffle=True, random_state=1)
    
    return X_train, X_test, y_train, y_test

Nel nostro esempio LBP viene eseguito Points=24 e Raggio=8. Il blocco un quadrato di lato 16 (dims) con sovrapposizione a scorrimento di 8 in 8 (shift).

In [6]:
X_train, X_test, y_train, y_test = load_lbp_features (base_path = './Esercitazioni_04_notebook/Esercitazioni_04/classes/')

Le funzioni successive bow_features e norm_disct_as lavorano concatenatamente. Dopo aver normalizzato le features si crea un dizionario tramite k-means. Ogni feature di ogni immagine verrà associata ad uno dei centroidi trovati. Si ottiene quindi che le features divengono l'istogramma normalizzato delle bag of words per ogni immagine.

In [7]:
def bow_features(feats, dictionary, nwords):
    quantized = dictionary.predict(feats)
    t = np.bincount(quantized, minlength=nwords)
    return t

def norm_dict_as(X_train, X_test, nwords, normalize=True, eps=0.001):
   
    #normalizzazione features
    X_train_stack = np.zeros((0,X_train[0].shape[1]), dtype=np.float32)
    for t in X_train:
        X_train_stack = np.concatenate((X_train_stack, t))
        
    if normalize:
        X_train_mean = X_train_stack.mean(axis=0)
        X_train_std = X_train_stack.std(axis=0)
        X_train = [(t - X_train_mean + eps)/(X_train_std + eps) for t in X_train]
        X_test = [(t - X_train_mean + eps)/(X_train_std + eps) for t in X_test]
        X_train_stack = (X_train_stack - X_train_mean + eps)/(X_train_std + eps)
        
    #creazione dizionario    
    dictionary = MiniBatchKMeans(n_clusters=nwords)
    dictionary.fit(X_train_stack)
    
    #assegnamento words
    X_train = [bow_features(f, dictionary, nwords) for f in X_train]
    X_test = [bow_features(f, dictionary, nwords) for f in X_test]
    X_train = [hist/hist.sum() for hist in X_train]
    X_test = [hist/hist.sum() for hist in X_test]
    return X_train, X_test

Troviamo quindi un dizionario con nwords = 300 e le features finali.

In [8]:
X_train2, X_test2 = norm_dict_as (X_train, X_test, 300)

Alleniamo una SVM 5 Folds Cross Validation a partire dei valori per i parametri C e gamma generici per poi ricercare più centratamente sui valori ottenuti.

In [9]:
param_grid = {'C': [ 1, 5, 10, 50, 100, 500],
          'gamma': [ 1, 5, 10, 50, 100, 500], } 

clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid, cv=5, n_jobs=-1)

t2 = time()
clf = clf.fit(X_train2, y_train)
print("Addestramento completato in %0.3fs" % (time() - t2))

print("Migliore combinazione di parametri:")
print(" C: "+str(clf.best_estimator_.C))
print(" gamma: "+str(clf.best_estimator_.gamma))

Addestramento completato in 34.271s
Migliore combinazione di parametri:
 C: 1
 gamma: 500


In [14]:
param_grid = {'C': [ 0.5, 1, 1.5, 2, 2.5],
          'gamma': [ 300, 400, 500, 600, 700], } 

clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid, cv=5, n_jobs=-1)

t2 = time()
clf = clf.fit(X_train2, y_train)
print("Addestramento completato in %0.3fs" % (time() - t2))

print("Migliore combinazione di parametri:")
print(" C: "+str(clf.best_estimator_.C))
print(" gamma: "+str(clf.best_estimator_.gamma))

Addestramento completato in 15.109s
Migliore combinazione di parametri:
 C: 1.5
 gamma: 300


I valori migliori sono risultati essere quindi C=1.5 e gamma=300.
Riportiamo i risultati sul test nel seguente report. 
A seguito la relativa matrice di confusione e accuracy= 0.7075

In [12]:
y_pred = clf.predict(X_test2)

print("Report di classificazione:")
print(classification_report(y_test, y_pred))

Report di classificazione:
             precision    recall  f1-score   support

          0       0.68      0.76      0.72       199
          1       0.74      0.65      0.69       201

avg / total       0.71      0.71      0.71       400



In [13]:
print("Matrice di confusione:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

print("Accuratezza media= " + str(accuracy_score(y_test, y_pred)))

Matrice di confusione:
[[152  47]
 [ 70 131]]
Accuratezza media= 0.7075
